In [92]:
## Things to add

# 1: Splitting functionality
# 2: Another game (i.e. Texas Hold 'em)
# 3: Multiplayer
# 4: Add computer player

## Store each card in a tuple (2, "Spades")<br/>Game would be easiest to build using classes<br/>Design the game as one normally would play Blackjack<br/>Be creative! You have creative control

In [93]:
# Hold all cards in a deck class. Object should handle things specific to deck, such as shuffling, creating deck, etc.
# Deck class should not have draw card function, as that is gameplay specific

# Imports

In [94]:
import random
from IPython.display import clear_output
import time

# Objects necessary for game play

In [98]:
class Deck():
    ''' Holds all control of creating and shuffling deck, not game play functions like dealing '''
    def __init__(self, numDecks):
        self.total = 52 * numDecks
        self.__suits = ("Spades", "Hearts", "Diamonds", "Clubs")
        self.numDecks = numDecks
        self.cards = []
        
    def createDeck(self):
        num = 0
        # Create as many decks as the player wishes, or we specify for game (3 for real game of BJ)
        while num < self.numDecks:
            for i in range(2, 15):
                for j in range(4):
                    if i == 11:
                        self.cards.append(("J", self.__suits[j]))
                    elif i == 12:
                        self.cards.append(("Q", self.__suits[j]))
                    elif i == 13:
                        self.cards.append(("K", self.__suits[j]))
                    elif i == 14:
                        self.cards.append(("A", self.__suits[j]))
                    else:
                        self.cards.append((i, self.__suits[j]))
            num += 1
        return self.cards
    
    def shuffleDeck(self):
        random.shuffle(self.cards)
        return self.cards
    

class BlackJack(Deck):
    ''' The game play itself, game name, instructions, game play functions '''
    def __init__(self, numDecks, game):
        super().__init__(numDecks)
        self.game = game
        
    def showInstr(self):
        print("Welcome to BlackJack!!")
        print("======================")
        print("How to play:")
        print("1) Have a higher hand than the dealer to win.")
        print("2) Going over 21 is called a 'bust' and you lose automatically.")
        print("3) The dealer must hit on any hand lower than 17.")
        print("4) If you and the dealer tie, it's a 'push' and neither you nor the dealer loses money.")
        print("5) Type 'hit' to receive a card from the deck on your turn.")
        print("6) Type 'stay' to end your turn.")
        print("7) Type 'double' to double down (once doubling down, your turn ends after receiving card).")
        print("8) Type 'split' to split the cards in your hand (only allowed on pairs).")
        print("9) Each player begins with $500")
        print("10) The player with the highest amount during the session will be displayed when you leave the table.")
        print("11) Press 'q' to quit at any time.")
        print("\nGood Luck!!")
        
    def draw(self):
        card = self.cards[0]
        del self.cards[0]
        return card
    
    def getCardCount(self):
        return len(self.cards)
    
    def dealCards(self, players):
        ''' Deals initial cards at beggining of round '''
        # Players passed in as a list
        for player in players:
            player.addToHand(self.draw())
            player.addToHand(self.draw())
        
    def calcTotal(self, hand):
        total = 0
        for card in hand:
            total += card[0]
        return total
        
class Player():
    ''' Keep track of player names, money, hand, etc. '''    
    def __init__(self, name):
        self.name = name
        self.hand = []
        self.turn_done = False
        self.double_down = False
        self.currency = 500
        self.highest_amt = 500
        self.bet = 0

    def getName(self):
        return self.name
    
    def setName(self, name):
        self.name = name
    
    def addToHand(self, card):
        self.hand.append(card)
        
    def getPlayerHand(self):
        return self.hand
    
    def updateBet(self, amount):
        self.bet = amount
        
    def getBet(self):
        return self.bet
        
    def updateCurrency(self, result):
        if result == "LOST":
            self.currency -= self.bet
        elif result == "PUSH":
            return
        elif result == "WON":
            self.currency += self.bet
            if self.currency > self.highest_amt:
                self.highest_amt = self.currency
        
    def getCurrency(self):
        return self.currency        
    
    def getHighestTotal(self):
        return self.highest_amt
        
    def calcTotal(self):
        total = 0
        has_ace = False
        for card in self.hand:
            if card[0] == 'J' or card[0] == 'Q' or card[0] == 'K':
                total += 10
            elif card[0] == 'A':
                has_ace = True
                if total + 11 > 21:
                    total += 1
                else:
                    total += 11
            else:
                total += int(card[0])
            
        # Check again for Aces and change them to 1 if necessary
        if total > 21 and has_ace == True:
            for card in self.hand:
                if card[0] == 'A':
                    total -= 10
        
        return total
    
    def dealerTotal(self):
        total = 0
        if self.hand[1][0] == 'J' or self.hand[1][0] == 'Q' or self.hand[1][0] == 'K':
            total += 10
        elif self.hand[1][0] == 'A':
            total += 11
        else:
            total += int(self.hand[1][0])
        return total
    
    def printHand(self, dealerHand = False):
        print("{}: Currency: {}, Bet: {}".format(self.name, self.getCurrency(), self.getBet()))
        print("========")
        if self.name == "Dealer" and len(self.hand) <= 2 and dealerHand == False:
            print("- of -")
            print("{} of {}".format(self.hand[1][0], self.hand[1][1]))
            print("Total = {}\n".format(self.dealerTotal()))
        else:
            for card in self.hand:
                print("{} of {}".format(card[0], card[1]))
            print("Total = {}\n".format(self.calcTotal()))
    
    def clearHand(self):
        self.hand = []

# Functions (not in classes)

In [99]:
# main dealer turn function, loops until dealer hand is complete
def dealerTurn(dealer, players, game):
    # Show dealer second card, pause..
    clear_output()
    dealer.printHand(True)
    player1.printHand()
    time.sleep(2)
            
    if player1.turn_done == False:
        while True:
            # have dealer hit or stay if any players left
            if dealer.calcTotal() < 17:
                time.sleep(2)
                clear_output()
                dealer.addToHand(game.draw())
                # if hit, display again
                for player in players:
                    player.printHand()
            else:
                break
                
def playerTurn(player1, game, players):
    # loop until they stay
    while True:
        # ask choice to hit or stay
        ans = input("Would you like to 'hit' or 'stay'? ")
        clear_output()
        if ans.lower() == 'stay':
            break
        # if hit, display again
        elif ans.lower() == 'hit':
            player1.addToHand(game.draw())
        elif ans.lower() == 'double':
            if player1.getCurrency() >= player1.bet * 2:
                player1.double_down = True
                player1.addToHand(game.draw())
                player1.bet *= 2
            else:
                print("Sorry you cannot double down, you do not have the funds.")

        # display cards
        for player in players:
            player.printHand()

        # check if player went over
        if player1.calcTotal() > 21:
            player1.turn_done = True
            break
        elif player1.double_down == True:
            break
            
def calculateWinner(dealer, player1):
    # calculate the winner and display message based on result
    # give money based on result
    if player1.turn_done == True:
        print("Sorry, but you went over, you lose!")
        player1.updateCurrency("LOST")
    elif dealer.calcTotal() <= 21 and dealer.calcTotal() > player1.calcTotal():
        print("The dealer beat you, you lose!")
        player1.updateCurrency("LOST")
    elif dealer.calcTotal() == player1.calcTotal():
        print("It's a push!")
        player1.updateCurrency("PUSH")
    else:
        print("You won! Congratulations you beat the dealer.")
        player1.updateCurrency("WON")
        
def resetGame(players):
    # reset all values and hand
    for player in players:
        player.clearHand()
        player.turn_done = False
        player.double_down = False
        
def handleBet(player1):
    while True:
        print("You have ${}".format(player1.getCurrency()))
        bet_num = input("How much would you like to bet? ")
        bet_num = int(bet_num)
        if bet_num <= player1.getCurrency() and bet_num > 0:
            player1.updateBet(bet_num)
            break
        else:
            clear_output()
            print("Sorry, but that is an incorrect amount.")
            
def playAgain():
    print("You have ${}".format(player1.getCurrency()))
    ans = input("Would you like to play again? ")
    if ans.lower() == 'no' or ans.lower() == 'q':
        clear_output()
        print("Thanks for playing! You're highest total amount was: ${}".format(player1.getHighestTotal()))
        time.sleep(5)
        return True

# Main game loop

In [104]:
# main outer - shows instr., prompts to push enter to play or q to quit
while True:
    clear_output()
    game = BlackJack(3, "BlackJack")
    game.showInstr()
    ans = input("Press enter to play or 'q' to quit! ")
    ans = ans.lower()
    
    # quit otherwise play
    if ans == 'q':
        break
    else:
        # game begins, create new game, shuffle cards, initialize deck
        game = BlackJack(3, "BlackJack")
        game.createDeck()
        game.shuffleDeck()
        
        # initalize players for now, later add ability to have multiple players
        dealer = Player("Dealer")
        player1 = Player("Player 1")
        players = [dealer, player1]
        
        # inner loop that has game flow itself
        while True:
            clear_output()
            
            # remove all cards from hand and reset attributes
            resetGame(players)
                
            # ask for bet
            handleBet(player1)
            
            clear_output() 
            
            # deal cards
            game.dealCards(players)
            
            # display cards
            for player in players:
                player.printHand()            
            
            # Player phase
            playerTurn(player1, game, players)
            
            # Dealer phase
            dealerTurn(dealer, players, game)

            # calculate winner
            calculateWinner(dealer, player1)
            
            if player1.getCurrency() <= 0:
                print("Sorry you lost all your money! Your max amount was ${}.".format(player1.getHighestTotal()))
                time.sleep(5)
                break
            # ask to play again, break loop if return = True
            again = playAgain()
            if again == True:
                break

Welcome to BlackJack!!
How to play:
1) Have a higher hand than the dealer to win.
2) Going over 21 is called a 'bust' and you lose automatically.
3) The dealer must hit on any hand lower than 17.
4) If you and the dealer tie, it's a 'push' and neither you nor the dealer loses money.
5) Type 'hit' to receive a card from the deck on your turn.
6) Type 'stay' to end your turn.
7) Type 'double' to double down (once doubling down, your turn ends after receiving card).
8) Type 'split' to split the cards in your hand (only allowed on pairs).
9) Each player begins with $500
10) The player with the highest amount during the session will be displayed when you leave the table.
11) Press 'q' to quit at any time.

Good Luck!!
Press enter to play or 'q' to quit! q
